# Logistic Regression

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
X_train, y_train = train.iloc[:, :-4], train.iloc[:, -4:]

In [6]:
lr = LogisticRegression(multi_class="multinomial")
lr.fit(X_train, y_train)

ValueError: y should be a 1d array, got an array of shape (160, 4) instead.